In [2]:
%load_ext autoreload
%autoreload 2

# Import library
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from IPython.display import display
from glob import glob
import cv2


# Import dependency
from src.preprocessing.generate_metadata import discover_wsi
from src.preprocessing.xml_to_mask import get_mask
from src.preprocessing.annotation_utils import resolve_annotation_path
from src.preprocessing.extract_patches import extract_patches
from src.preprocessing.load_wsi import load_wsi
#from src.train.train_phase1 import train_phase1


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Configuration
BASE_DIR = 'data'
SOURCES = [
    'Yale_HER2_cohort',
    'Yale_trastuzumab_response_cohort',
    'TCGA_BRCA_Filtered'
]
OUTPUT_CSV = 'outputs/index/wsi_index.csv'

In [4]:
log_dir = 'outputs/preprocessing/logs'
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, 'preprocessing.log')

def log(msg):
    print(msg)
    with open(log_path, 'a') as f:
        f.write(msg + '\n')

In [5]:
csv_path = discover_wsi(
    base_dir=BASE_DIR, 
    sources=SOURCES, 
    output_path=OUTPUT_CSV
)

# Load and display the results
df = pd.read_csv(csv_path)
display(df.head(50))

Processing sources: 100%|██████████| 3/3 [00:00<00:00, 79.37it/s]


,wsi_path,slide_id,slide_name,annotation_name,annotation_path
0,data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs,Her2Neg_Case_01,Her2Neg_Case_01.svs,Her2Neg_Case_01.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
1,data/Yale_HER2_cohort/SVS/Her2Neg_Case_02.svs,Her2Neg_Case_02,Her2Neg_Case_02.svs,Her2Neg_Case_02.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
2,data/Yale_HER2_cohort/SVS/Her2Neg_Case_03.svs,Her2Neg_Case_03,Her2Neg_Case_03.svs,Her2Neg_Case_03.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
3,data/Yale_HER2_cohort/SVS/Her2Neg_Case_04.svs,Her2Neg_Case_04,Her2Neg_Case_04.svs,Her2Neg_Case_04.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
4,data/Yale_HER2_cohort/SVS/Her2Neg_Case_05.svs,Her2Neg_Case_05,Her2Neg_Case_05.svs,Her2Neg_Case_05.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
5,data/Yale_HER2_cohort/SVS/Her2Neg_Case_06.svs,Her2Neg_Case_06,Her2Neg_Case_06.svs,Her2Neg_Case_06.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
6,data/Yale_HER2_cohort/SVS/Her2Neg_Case_07.svs,Her2Neg_Case_07,Her2Neg_Case_07.svs,Her2Neg_Case_07.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
7,data/Yale_HER2_cohort/SVS/Her2Neg_Case_08.svs,Her2Neg_Case_08,Her2Neg_Case_08.svs,Her2Neg_Case_08.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
8,data/Yale_HER2_cohort/SVS/Her2Neg_Case_09.svs,Her2Neg_Case_09,Her2Neg_Case_09.svs,Her2Neg_Case_09.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
9,data/Yale_HER2_cohort/SVS/Her2Neg_Case_10.svs,Her2Neg_Case_10,Her2Neg_Case_10.svs,Her2Neg_Case_10.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...


In [6]:
for idx, row in tqdm(df.iterrows(), total=len(df), desc='Processing slides'):
    wsi_path = row['wsi_path']
    # Resolve annotation path using helper (handles pandas NA, relative paths, and glob fallback)
    annotation_path = resolve_annotation_path(row.get('annotation_path', None), wsi_path, base_dir=BASE_DIR)
    if not annotation_path:
        log(f"Skipping slide without annotation: {wsi_path}")
        continue

    log(f"Processing slide: {wsi_path} with annotation: {annotation_path}")
    try:
        mask = get_mask(annotation_path, wsi_path)
    except Exception as e:
        log(f"Failed to generate mask for {wsi_path}: {e}")
        continue
    if mask is None:
        log(f"No mask generated for {wsi_path}")
        continue

    # At this point mask should be a 2D uint8 array (0 or 255)
    log(f'Mask shape: {mask.shape}')

    # Load WSI using the wrapper which prefers CuCIM when available
    try:
        wsi_slide = load_wsi(wsi_path)
    except Exception as e:
        log(f"Failed to load WSI ({wsi_path}): {e}")
        continue
    if wsi_slide is None:
        log(f"Failed to load WSI: {wsi_path}")
        continue

    # Log which backend the loader selected (cucim or openslide)
    backend = getattr(wsi_slide, 'backend', None)
    log(f'Loaded WSI backend: {backend}')

    # Extract patches (extract_patches can optionally save patches to disk)
    slide_base = os.path.splitext(os.path.basename(wsi_path))[0]
    out_dir_patches = os.path.join('outputs', 'patches', slide_base)
    try:
        patches = extract_patches(
            wsi_slide,
            mask=mask,
            size=512,
            stride=512,
            save_dir=out_dir_patches,
            save_prefix=slide_base,
            save_format='png'
        )
    except Exception as e:
        log(f"Failed to extract patches for {wsi_path}: {e}")
        continue

    saved = sum(1 for p in patches if p.get('path'))
    log(f'Extracted {len(patches)} patches from {wsi_path}; saved {saved} to {out_dir_patches}')

Processing slides:   0%|          | 0/471 [00:00<?, ?it/s]

Processing slide: data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/Yale_HER2_cohort/Annotations/Her2Neg_Case_01.xml
Mask shape: (45426, 41888)
Loaded WSI backend: cucim


Processing slides:   0%|          | 1/471 [00:06<52:14,  6.67s/it]

Extracted 155 patches from data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs; saved 155 to outputs/patches/Her2Neg_Case_01
Processing slide: data/Yale_HER2_cohort/SVS/Her2Neg_Case_02.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/Yale_HER2_cohort/Annotations/Her2Neg_Case_02.xml
Mask shape: (28692, 31416)
Loaded WSI backend: cucim


Processing slides:   0%|          | 2/471 [00:10<39:24,  5.04s/it]

...

Extracted 8405 patches from data/TCGA_BRCA_Filtered/SVS/TCGA-BH-A0DZ-01Z-00-DX1.138BAEC2-589E-4960-B94E-DF48DDAA5490.svs; saved 8405 to outputs/patches/TCGA-BH-A0DZ-01Z-00-DX1.138BAEC2-589E-4960-B94E-DF48DDAA5490
Processing slide: data/TCGA_BRCA_Filtered/SVS/TCGA-BH-A0E2-01Z-00-DX1.5F6FA19C-D59F-45BE-80CE-F738CAB1EF0B.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/TCGA_BRCA_Filtered/Annotations/TCGA-BH-A0E2-01Z-00-DX1.xml
Mask shape: (77417, 133104)
Loaded WSI backend: cucim


Processing slides:  81%|████████  | 381/471 [9:55:05<13:38:28, 545.65s/it]

Extracted 7766 patches from data/TCGA_BRCA_Filtered/SVS/TCGA-BH-A0E2-01Z-00-DX1.5F6FA19C-D59F-45BE-80CE-F738CAB1EF0B.svs; saved 7766 to outputs/patches/TCGA-BH-A0E2-01Z-00-DX1.5F6FA19C-D59F-45BE-80CE-F738CAB1EF0B
Processing slide: data/TCGA_BRCA_Filtered/SVS/TCGA-BH-A0EE-01Z-00-DX1.872EA586-25C8-4835-A138-152DFA1EBF30.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/TCGA_BRCA_Filtered/Annotations/TCGA-BH-A0EE-01Z-00-DX1.xml
Mask shape: (93788, 125444)
Loaded WSI backend: cucim


Processing slides:  81%|████████  | 381/471 [10:12:26<2:24:40, 96.45s/it] 


KeyboardInterrupt: 

# Phase 1 — Train ResNet-50 (module)
This trains a patch-level HER2 classifier using ResNet-50.

Inputs: two CSV files with columns `path` and `label` (0 = negative, 1 = positive).
Outputs:
- Best model: `outputs/phase1/models/model_phase1.pth`
- Logs/metrics: `outputs/phase1/logs`

In [ ]:
# CSV Detail: path (image path), label (0=negative, 1=positive)
CFG = {
    'train_csv': 'outputs/patches_index_train.csv',
    'val_csv': 'outputs/patches_index_val.csv',
    'output_dir': 'outputs/phase1',
    'pretrained': True,
    'input_size': 512,
    'batch_size': 32,
    'num_workers': 4,
    'epochs': 10,
    'lr': 1e-4,
    'weight_decay': 1e-4,
    'label_col': 'label',
    'path_col': 'path',
    'save_best_by': 'auc',
    'seed': 42,
}

results = train_phase1(CFG)
print('Best model:', results['best_model_path'])
print('Logs dir:', results['logs_dir'])